In [2]:
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import cv2
from preprocessing.preprocess import preprocess
from segmentation.segmentation import Segmentation
from utils import _rmlead

In [4]:
path = '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Segmentation_Neutrophils/'
in_folder = 'images used by SK 20211228_Sample_cohort_with_infiltration'
out_folder = 'masks'

In [5]:
df = pd.DataFrame(columns = ['Sample', 'FoV', 'img'])

for i,f in enumerate(os.listdir(os.path.join(path, in_folder))):
    #df.loc[i] = [f.split(' ')[0], _rmlead(f.split('-')[2]), cv2.imread(os.path.join(path, in_folder, f), cv2.IMREAD_GRAYSCALE)]
    df.loc[i] = [f.split(' ')[0], _rmlead(f.split('-')[2]), cv2.imread(os.path.join(path, in_folder, f))]


In [6]:
segment = Segmentation(option='cellpose', mpp=0.1725)

masks, _, _, _ = segment(df['img'][0:2])

x=0

Using Cellpose for segmentation!
Predicting using Cellpose


In [6]:
import tifffile
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from deepcell.utils.plot_utils import make_outline_overlay, create_rgb_image
from skimage.segmentation import find_boundaries

def make_outline_overlay(rgb_data, predictions):

    boundaries = np.zeros_like(rgb_data)
    overlay_data = np.copy(rgb_data)

    for img in range(predictions.shape[0]):
        boundary = find_boundaries(predictions[img, ..., 0], connectivity=1, mode='thick')
        boundaries[img, boundary > 0, :] = 1

    overlay_data[boundaries > 0] = 1

    return overlay_data

def create_outline(im, mask):
    
    rgb = create_rgb_image(im, ["blue"])
    outline = make_outline_overlay(rgb, mask)
    
    return outline.squeeze()


path_2_masks = '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Segmentation_Neutrophils/masks/Johannes_seg/segmentation_results'
path_2_images = '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Segmentation_Neutrophils/images used by SK 20211228_Sample_cohort_with_infiltration'
out_path = '/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Segmentation_Neutrophils/masks/outline'

for f in os.listdir(path_2_masks):
    mask = tifffile.imread(os.path.join(path_2_masks, f))
    img_name = f.split('_')[0]
    img = cv2.imread(glob.glob(path_2_images + '/*{}*'.format(img_name))[0], 0)

    mask = np.expand_dims(np.expand_dims(mask, -1), 0)
    img = np.expand_dims(np.expand_dims(img, -1), 0)
    o = create_outline(img, mask)
    plt.rcParams["figure.figsize"] = 20,20
    plt.imsave(os.path.join(out_path, f"{img_name}_outline.tiff"), o)

